In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
# Transformação para normalizar os dados
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Carregar o dataset MNIST
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader para carregar os dados em batches
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instanciar o modelo
model = Net()


In [ ]:
from futurelabs.lab.project import Project

project_log = Project(
        project_name="MNIST",
        work_folder="logs",
        laboratory_name="lab 1",
    ).log()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from futurelabs.lab.chart import Type
import polars as pl


log_class = project_log.new_logger(
        section_name="Classificação",
        description="Aqui está sendo monitorando o gradiente de 3 camadas do modelo",
        chart_type=Type.Classification
    )

def train(model, device, train_loader, optimizer, epoch):
    model.train()

    outputs_list = []
    target_list = []

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # print(output)
        outputs_list.append(output)
        outputs_list.append(target)

        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            
    log_class.log_classification("Classificação",target_list, outputs_list, epoch)
    # print("Gerou!!!")
    # df = pl.DataFrame(
    #         {
    #             "step": epoch,
    #             "real_label": target_list,
    #             "predicted_label": outputs_list,
    #         }
    #     )
    # 
    # 
    # df.write_parquet("data.parquet")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
